In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats/test1.zip
/kaggle/input/dogs-vs-cats/train.zip
/kaggle/input/dogs-vs-cats/sampleSubmission.csv


In [2]:
import zipfile
import os
# zip file handler  
zip = zipfile.ZipFile('/kaggle/input/dogs-vs-cats/train.zip')
zip.extractall('/kaggle/temp/')

# list available files in the container
#print (zip.namelist())
'''import shutil
shutil.rmtree('/kaggle/working/train')'''

"import shutil\nshutil.rmtree('/kaggle/working/train')"

In [3]:
filenames = os.listdir('/kaggle/temp/train') 
'''example: ['dog.890.jpg',
 'dog.1178.jpg',
 'dog.7845.jpg',
 'dog.4632.jpg',
 'cat.3660.jpg']'''
import re

category=[]
id = []
for name in filenames:
    id.append(name.split('.')[1])
    category.append(name.split('.')[0])
    '''if re.match(r"dog.*", name):
        category.append(1)
    else: 
        category.append(0)'''
df = pd.DataFrame.from_dict({'id':id,'filename':filenames,'category':category})
df.head()

,id,filename,category
0,890,dog.890.jpg,dog
1,1178,dog.1178.jpg,dog
2,7845,dog.7845.jpg,dog
3,4632,dog.4632.jpg,dog
4,3660,cat.3660.jpg,cat


In [4]:
df.groupby('category').count()

,id,filename
category,,
cat,12500,12500
dog,12500,12500


In [5]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3, stratify=df.category)

In [6]:
train.groupby('category').count()

,id,filename
category,,
cat,8750,8750
dog,8750,8750


In [7]:
test.groupby('category').count()

,id,filename
category,,
cat,3750,3750
dog,3750,3750


In [8]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_dataframe(train,directory='/kaggle/temp/train',x_col="filename", y_col="category",
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

# Preprocessing the Test set
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_dataframe(test,directory='/kaggle/temp/train',x_col="filename", y_col="category",
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 17500 validated image filenames belonging to 2 classes.
Found 7500 validated image filenames belonging to 2 classes.


In [9]:
print(train.shape)
print(test.shape)

(17500, 3)
(7500, 3)


In [10]:
# Building the CNN
from keras.models import Model,Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense,Flatten
# Initialising the CNN
cnn = tf.keras.models.Sequential()

# Step 1 - Convolution
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(Dropout(0.2))
# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
cnn.add(Dropout(0.2))
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(Dropout(0.2))
# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Part 3 - Training the CNN

# Compiling the CNN
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3
)
# Training the CNN on the Training set and evaluating it on the Test set
history = cnn.fit(x = training_set, validation_data = test_set, epochs = 25, callbacks=[es])
history.history['val_loss']

Epoch 1/25
547/547 [==============================] - 103s 187ms/step - loss: 0.6938 - accuracy: 0.5050 - val_loss: 0.6870 - val_accuracy: 0.5000
Epoch 2/25
547/547 [==============================] - 102s 187ms/step - loss: 0.6780 - accuracy: 0.5813 - val_loss: 0.6362 - val_accuracy: 0.6485
Epoch 3/25
547/547 [==============================] - 102s 186ms/step - loss: 0.6485 - accuracy: 0.6197 - val_loss: 0.5901 - val_accuracy: 0.6897
Epoch 4/25
547/547 [==============================] - 102s 186ms/step - loss: 0.5985 - accuracy: 0.6781 - val_loss: 0.5576 - val_accuracy: 0.6952
Epoch 5/25
547/547 [==============================] - 102s 186ms/step - loss: 0.5604 - accuracy: 0.7106 - val_loss: 0.5000 - val_accuracy: 0.7556
Epoch 6/25
547/547 [==============================] - 100s 183ms/step - loss: 0.5229 - accuracy: 0.7368 - val_loss: 0.4577 - val_accuracy: 0.7817
Epoch 7/25
547/547 [==============================] - 101s 185ms/step - loss: 0.4959 - accuracy: 0.7585 - val_loss: 0.4698 -

[0.6870089173316956,
 0.6361600160598755,
 0.5900741815567017,
 0.5576307773590088,
 0.5000194311141968,
 0.45769673585891724,
 0.4697992503643036,
 0.42338651418685913,
 0.4459749460220337,
 0.4049776792526245,
 0.3999503254890442,
 0.3932342529296875,
 0.4355040490627289,
 0.42448484897613525,
 0.37421467900276184,
 0.35832956433296204,
 0.3891378343105316,
 0.3513873517513275,
 0.3516543209552765,
 0.3257339894771576,
 0.34272629022598267,
 0.3342386782169342,
 0.3556255102157593]

In [11]:
zip = zipfile.ZipFile('/kaggle/input/dogs-vs-cats/test1.zip')
zip.extractall('/kaggle/temp/')

# list available files in the container
#print (zip.namelist())

In [12]:
filenames = os.listdir('/kaggle/temp/test1') 
'''example: '''
import re
id = []
for name in filenames:
    id.append(name.split('.')[0])
    
df = pd.DataFrame.from_dict({'id':id,'filename':filenames})
df.head()

,id,filename
0,10435,10435.jpg
1,6790,6790.jpg
2,4644,4644.jpg
3,1835,1835.jpg
4,1102,1102.jpg


In [13]:
predict_datagen = ImageDataGenerator(rescale = 1./255)
predict_set = predict_datagen.flow_from_dataframe(df,directory='/kaggle/temp/test1',x_col="filename", y_col=None,
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = None,
                                            shuffle=False)

Found 12500 validated image filenames.


In [14]:
pred_out=cnn.predict_generator(predict_set,steps = df.shape[0])

/opt/conda/lib/python3.7/site-packages/tensorflow/python/keras/engine/training.py:1905: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  warnings.warn('`Model.predict_generator` is deprecated and '


In [15]:
training_set.class_indices

{'cat': 0, 'dog': 1}

In [16]:
labels= [1 if output > 0.5 else 0 for output in pred_out]
#labels

In [17]:
df['label']=labels

In [18]:
df.drop(columns=['filename'],inplace=True)


In [19]:
df.head()

,id,label
0,10435,0
1,6790,1
2,4644,0
3,1835,0
4,1102,0


In [20]:
df.to_csv('submission1.csv', index=False)